

# AMLS Assignment Draft
## Task A_3: CNN on BreastMNIST Dataset Validating Learning Rate

Explore range of Learning Rates for CNN based model on the BreastMNIST dataset.

## Import libraries
The required libraries for this notebook include numpy, matplotlib, tensorflow, medmnist and also a custom built dataload and utility library.

In [1]:
## import libraries
## first enable autoreload during development so latest (new) version local code library is reloaded on execution 
## can be commented out when local code development not happening to avoid overhead
%load_ext autoreload
%autoreload 2
    
import io
import time                          ## to enable delays between file saves
import numpy as np                 
import matplotlib.pyplot as plt      ## for graphing
import pandas as pd                  ## to enable data structure changes
## import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, Hinge

## removed MedMNIST as only used in AMLS common library

## local code library
import AMLS_common as ac

## Set base parameters
Including hyper parameters and data set specifics

In [2]:
## set up lists and parameters
test_list     = []
run_list      = []
## use these lists of values to grid test hyper parameter sensitivity                
epochs_list   = [30,40]                         ## set of epochs to run for
bs_list       = [128]                           ## dataset batch size
lr_list       = [0.01, 0.001, 0.0001]           ## learning rates
filter_list   = [16,32,64]                      ## main filter sizes to use
ly_list       = [2,3]                         ## number of covolution layers
dr_list       = [0.1,0.4]               ## selected dropout rates
loss_list     = ['binary_crossentropy'] ## loss functions to use
optimise_list = ['Adam']        ## optimisation functions
st_list       = [1,2]
for ep in epochs_list:
    for bs in bs_list:
        for lr in lr_list:
            for fi in filter_list:
                for ly in ly_list:
                    for dr in dr_list:
                        for ls in loss_list:
                            for op in optimise_list:
                                for st in st_list:
                                    parameter = ac.HyperParameters(learning_rate=lr, 
                                                                   batch_size=bs, 
                                                                   num_epochs=ep, 
                                                                   num_filter=fi,
                                                                   layers=ly,
                                                                   dropout_rate=dr,
                                                                   strides=st,
                                                                   padding="same",
                                                                   optimise=op,
                                                                   loss=ls)          
                                    test_list.append([parameter])
## reshape parameters into a test grid that can be read using for loop
test_grid = [hp for sublist in test_list for hp in sublist]
print("test cases:",len(test_grid))

test cases: 144


In [3]:
## control and environment (e.g. verbose) parameters
filebase   = "metrics/"          # the folder to store the metrics and summary files generated for each run
verbose    = 0                   # if value equals 1 then print additional process information in steps below

## Load and preprocess the BreastMNIST Data
We load the dataset.

In [4]:
## Loading the data file using custom MedMINST loader
data_flag  = 'breastmnist'       ## defines which dataset to load
result_set = ac.medMNIST_load(data_flag,128)

## check that the loader returned data correctly and then split out
if result_set != []:
    train_dataset = result_set[0]
    test_dataset  = result_set[1]
    val_dataset   = result_set[2]

if verbose == 1:
    print("\nSummary metrics for validation dataset")
    print("type:",type(val_dataset))
    print("length:",len(val_dataset))
    print("shape:",val_dataset)

Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\johnc\.medmnist\breastmnist.npz


## Fit the model

Using each Hyperparameter in turn from a superset

In [5]:
## Create instances of the dataclass from the list
for item in test_grid:
    ## Define the model which is then run for all learning rates in set
    print("Run with",item)
    print(item.num_epochs)
    
    ## initialise tqdm callback
    tqdm_callback = ac.TqdmEpochProgress(total_epochs=item.num_epochs)
    
    ## Simple CNN model to support Learning Rate analysis
    ## added desired number of layers
    if item.layers == 2:
        model = Sequential([
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter*2, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Dense(item.num_filter*4,activation=item.default_activation),              ## Fully connected layer
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])
    if item.layers == 3:
        model = Sequential([
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter*2, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Dense(item.num_filter*4, activation=item.default_activation),             ## Fully connected layer
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])
    if item.layers == 4:
        model = Sequential([
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation, input_shape=(28, 28, 1)),  ## Input layer
            Conv2D(item.num_filter*2, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            Conv2D(item.num_filter*2, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Conv2D(item.num_filter, (3, 3), padding=item.padding,\
                   activation=item.default_activation),                           ## Another convolution layer
            MaxPooling2D((2, 2),strides=item.strides),                            ## Reduce the feature maps
            Flatten(),                                                            ## Flatten
            Dropout(item.dropout_rate),                                           ## Drop out to reduce possible over fitting
            Dense(item.num_filter*4, activation=item.default_activation),             ## Fully connected layer
            Dense(1, activation='sigmoid')                                        ## Output layer for binary classification  
        ])    
        
    if verbose == 1:
        print(model.summary())
        
    ## Redirect the summary output to a string
    summary_string  = io.StringIO()
    model.summary(print_fn=lambda x: summary_string.write(x + "\n"))
    summary_content = summary_string.getvalue()
    summary_string.close()

    ## Compile the model
    model.compile(optimizer=item.optimise,                                                   
                  loss=item.loss,
                  metrics='acc')

    ## Fit the model
    history = model.fit(val_dataset, 
                        epochs=item.num_epochs, 
                        batch_size=item.batch_size, 
                        verbose=0,
                        callbacks = [tqdm_callback])

    ## Save results to files
    run_list.append(ac.hyper_process(history,summary_content,item))

print("Hyperparameter test run complete")

Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=1, padding='same', dropout_rate=0.1, layers=2, default_activation='relu')
30


Epoch Progress: 100%|██████████| 30/30 [00:04<00:00,  6.58epoch/s, loss=0.188, acc=0.923]


Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=2, padding='same', dropout_rate=0.1, layers=2, default_activation='relu')
30


Epoch Progress: 100%|██████████| 30/30 [00:02<00:00, 13.48epoch/s, loss=0.508, acc=0.731]


Run with HyperParameters(learning_rate=0.01, batch_size=128, num_epochs=30, optimise='Adam', loss='binary_crossentropy', num_filter=16, strides=1, padding='same', dropout_rate=0.4, layers=2, default_activation='relu')
30


...

Run with HyperParameters(learning_rate=0.0001, batch_size=128, num_epochs=40, optimise='Adam', loss='binary_crossentropy', num_filter=64, strides=1, padding='same', dropout_rate=0.4, layers=3, default_activation='relu')
40


Epoch Progress: 100%|██████████| 40/40 [00:33<00:00,  1.18epoch/s, loss=0.313, acc=0.833]


Run with HyperParameters(learning_rate=0.0001, batch_size=128, num_epochs=40, optimise='Adam', loss='binary_crossentropy', num_filter=64, strides=2, padding='same', dropout_rate=0.4, layers=3, default_activation='relu')
40


Epoch Progress: 100%|██████████| 40/40 [00:13<00:00,  2.90epoch/s, loss=0.339, acc=0.859]

Hyperparameter test run complete


In [11]:
best_run,run_df,best_run2 = ac.analyse_run(run_list," ",filebase)
print("\nRun satisfying both smallest min_loss and largest max_acc:")
print(best_run)
print("\n")
print(best_run2)

feature_importance,coef = ac.analyse_hyperparameters(run_df)
print("\nImpact of Hyperparameters on Accuracy (from Linear Regression):")
print(coef)
print("\nHyperparameter Importance for Accuracy (from Random Forest):")
print(feature_importance)

extracting

Run satisfying both smallest min_loss and largest max_acc:
    metrics_file summary_file  min_loss  max_acc  last_loss  last_acc  \
104                            0.016935      1.0   0.016935       1.0   

     var_loss   var_acc  learning_rate  batch_size  num_epochs  num_filter  \
104  0.043449  0.019872          0.001         128          40          32   

     strides padding  dropout_rate  layers optimise                 loss  \
104        1    same           0.1       2     Adam  binary_crossentropy   

    default_activation  
104               relu  


    metrics_file summary_file  min_loss  max_acc  last_loss  last_acc  \
104                            0.016935      1.0   0.016935       1.0   

     var_loss   var_acc  learning_rate  batch_size  num_epochs  num_filter  \
104  0.043449  0.019872          0.001         128          40          32   

     strides padding  dropout_rate  layers optimise                 loss  \
104        1    same           0.1      